# **5. MobileNetV3**

First of all, let's load the required libraries in order to run the code:

In [12]:
# Base libraries
import os

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Tensorflow
import tensorflow.keras as keras
from tensorflow import debugging as tfdbg
from tensorflow import device
from tensorflow.keras import optimizers
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# ResNet50
from tensorflow.keras.applications import MobileNetV3Large, MobileNetV3Small

# Own modules
from src.utils import drop_checkpoints, dataset_stats, plot_metric_curves

Now let's see how our data is structured:

In [13]:
# Root folder
base_dir = "./data"

In [14]:
# Train folder
train_dir = os.path.join(base_dir, "train")

# Validation folder
validation_dir = os.path.join(base_dir, "validation")

# Test folder
test_dir = os.path.join(base_dir, "test")

In [15]:
drop_checkpoints(base_dir)

Let's also see how many images there are for each class in the training, validation and test set.

In [16]:
dataset_stats(train_dir, validation_dir, test_dir)

Number of classes: 4
Existing classes: ['Baroque', 'Realism', 'Renaissance', 'Romanticism']

----------------------------------------
Number of images per class and dataset:
----------------------------------------
             Train  Validation  Test
Style                               
Baroque       4000         500   500
Realism       4000         500   500
Renaissance   4000         500   500
Romanticism   4000         500   500


We'll also create the directory, if not created yet, where the models will be saved:

In [17]:
# Create directory where to save the models created
models_dir = "./models"
os.makedirs(models_dir, exist_ok=True)

*Explain dropout and data augmentation, include reference to original paper*

# **5.1 MobileNetV3 with frozen convolutional base (just training classifier)**

## 5.1.1. Model structure

Let's first create the model structure:

Firstly, let's define the values of some hyperparameters:

In [18]:
# Define some hyperparameters' values

# Input shape
input_shape = (224, 224,  3)

# Batch_size and steps per epoch
training_size = sum([len(file) for path, folder, file in os.walk(train_dir)])
batch_size = 128
steps_per_epoch = training_size // batch_size
print(f"Steps per epoch: {steps_per_epoch}")

Steps per epoch: 125


In [51]:
# conv_base = MobileNetV3Large( 
#     include_top=False,
#     weights="imagenet",
#     input_shape=input_shape,
#     alpha=1.0,
#     pooling=None,
#     dropout_rate=0.2,
# )

conv_base = MobileNetV3Small( 
    include_top=False,
    weights="imagenet",
    input_shape=input_shape,
    alpha=1.0,
    pooling=None,
    dropout_rate=0.2,
)

# Create the model
model = models.Sequential()

# Add the convolutional base
model.add(conv_base)

# Add the classifier
model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.25))

model.add(layers.BatchNormalization())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.25))

# Output layer
model.add(layers.Dense(4, activation='softmax'))

# Freeze the convolutional base
model.layers[0].trainable = False

Once the structure of the base model has been defined, let's see exactly how many parameters it has in order to have a better idea of how flexible this model is:

In [52]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3small (Functiona  (None, 7, 7, 576)        939120    
 l)                                                              
                                                                 
 flatten_14 (Flatten)        (None, 28224)             0         
                                                                 
 batch_normalization_19 (Bat  (None, 28224)            112896    
 chNormalization)                                                
                                                                 
 dense_28 (Dense)            (None, 128)               3612800   
                                                                 
 dropout_20 (Dropout)        (None, 128)               0         
                                                                 
 batch_normalization_20 (Bat  (None, 128)            

We'll use Adam as our optimizer since it is the most popular optimizer right now, as well as versatile (i.e., it can be used in multiple contexts).

In [21]:
# optimizer = optimizers.SGD(learning_rate=0.1, momentum=0.9, weight_decay=0.0001)

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['acc']
)

## 5.1.2. Data preprocessing

In this case, we will include the Data Augmentation step to the model preprocessing step...

In [22]:
# Apply data augmentation to the training set
# https://towardsdatascience.com/exploring-image-data-augmentation-with-keras-and-tensorflow-a8162d89b844
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=(0.8, 1),
    zoom_range=[0.9, 1.1],
    horizontal_flip=True,
    fill_mode='nearest'
    )

# The data augmentation must not be used for the test set!
# All images will be rescaled by 1./255
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        # All images will be resized to the dimensions specified
        target_size=input_shape[:2],
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
        )

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        # All images will be resized to the dimensions specified
        target_size=input_shape[:2],
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
        )

Found 16000 images belonging to 4 classes.
Found 2000 images belonging to 4 classes.


Now let's take a look at the output of one of these generators (for instance, the training one):

In [23]:
for data_batch, labels_batch in train_generator:
    print('Data batch shape:', data_batch.shape)
    print('Labels batch shape:', labels_batch.shape)
    break

Data batch shape: (128, 224, 224, 3)
Labels batch shape: (128, 4)


*We can appreciate that...*

## 5.1.3. Training

Let's train the model:

We use [Early Stopping](https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/) to avoid *overfitting*, as well `ModelCheckpoint` to save the best model obtained during training:

In [24]:
# Model name and path
model_path = os.path.join("models", "mobile_frozen_model.h5")

In [25]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint(model_path, monitor='val_loss', 
                     mode='min', verbose=1, save_best_only=True)

In [26]:
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=10,
    callbacks = [es, mc]
)

Epoch 1/30
  4/125 [..............................] - ETA: 11:59 - loss: 3.0733 - acc: 0.2930

KeyboardInterrupt: 

Now let's load the best model found:

In [16]:
# load the saved model
saved_model = load_model(model_path)

## 5.1.4. Validation

Let's plot how the loss and the accuracy from both training and validations sets have evolved during the training process. 

In [17]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

In [ ]:
# Plot loss curves
plot_metric_curves(epochs, loss, val_loss, "darkcyan", "turquoise", "Loss")

In [ ]:
# Plot accuracy curves
plot_metric_curves(epochs, acc, val_acc, "darkcyan", "turquoise", "Accuracy")

In [27]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=input_shape[:2],
        batch_size=40,
        class_mode='categorical',
        shuffle=False
)

Found 2000 images belonging to 4 classes.


In [ ]:
model.evaluate(test_generator)

In [ ]:
saved_model.evaluate(test_generator)

*Comments about how those metrics have evolved...*

# **5.2 MobileNetV3 training classifier and last convolutional layer**

## 5.2.1. Model structure

Let's first create the model structure:

Firstly, let's define the values of some hyperparameters:

In [58]:
# Define some hyperparameters' values

# Input shape
input_shape = (224, 224,  3)

# Batch_size and steps per epoch
training_size = sum([len(file) for path, folder, file in os.walk(train_dir)])
batch_size = 128
steps_per_epoch = training_size // batch_size
print(f"Steps per epoch: {steps_per_epoch}")

Steps per epoch: 125


In [59]:
# conv_base = MobileNetV3Large( 
#     include_top=False,
#     weights="imagenet",
#     input_shape=input_shape,
#     alpha=1.0,
#     pooling=None,
#     dropout_rate=0.2,
# )

conv_base = MobileNetV3Small( 
    include_top=False,
    weights="imagenet",
    input_shape=input_shape,
    alpha=1.0,
    pooling=None,
    dropout_rate=0.2,
)
conv_base.summary()

Model: "MobilenetV3small"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_18 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_17 (Rescaling)       (None, 224, 224, 3)  0           ['input_18[0][0]']               
                                                                                                  
 Conv (Conv2D)                  (None, 112, 112, 16  432         ['rescaling_17[0][0]']           
                                )                                                                 
                                                                                   

In [60]:
# Freeze the convolutional base unless the last layer
conv_base.trainable = True

# Unfreeze the last layer
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'Conv_1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

# Create the model
model = models.Sequential()

# Add the convolutional base
model.add(conv_base)

# Add the classifier
model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.25))

model.add(layers.BatchNormalization())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.25))

# Output layer
model.add(layers.Dense(4, activation='softmax'))

Once the structure of the base model has been defined, let's see exactly how many parameters it has in order to have a better idea of how flexible this model is:

In [61]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3small (Functiona  (None, 7, 7, 576)        939120    
 l)                                                              
                                                                 
 flatten_17 (Flatten)        (None, 28224)             0         
                                                                 
 batch_normalization_25 (Bat  (None, 28224)            112896    
 chNormalization)                                                
                                                                 
 dense_37 (Dense)            (None, 128)               3612800   
                                                                 
 dropout_26 (Dropout)        (None, 128)               0         
                                                                 
 batch_normalization_26 (Bat  (None, 128)            

We'll use Adam as our optimizer since it is the most popular optimizer right now, as well as versatile (i.e., it can be used in multiple contexts).

In [62]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['acc']
)

## 5.2.2. Data preprocessing

In this case, we will include the Data Augmentation step to the model preprocessing step...

In [63]:
# Apply data augmentation to the training set
# https://towardsdatascience.com/exploring-image-data-augmentation-with-keras-and-tensorflow-a8162d89b844
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=(0.8, 1),
    zoom_range=[0.9, 1.1],
    horizontal_flip=True,
    fill_mode='nearest'
    )

# The data augmentation must not be used for the test set!
# All images will be rescaled by 1./255
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        # All images will be resized to the dimensions specified
        target_size=input_shape[:2],
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
        )

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        # All images will be resized to the dimensions specified
        target_size=input_shape[:2],
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
        )

Found 16000 images belonging to 4 classes.
Found 2000 images belonging to 4 classes.


Now let's take a look at the output of one of these generators (for instance, the training one):

In [64]:
for data_batch, labels_batch in train_generator:
    print('Data batch shape:', data_batch.shape)
    print('Labels batch shape:', labels_batch.shape)
    break

Data batch shape: (128, 224, 224, 3)
Labels batch shape: (128, 4)


*We can appreciate that...*

## 5.2.3. Training

Let's train the model:

We use [Early Stopping](https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/) to avoid *overfitting*, as well `ModelCheckpoint` to save the best model obtained during training:

In [65]:
# Model name and path
model_path = os.path.join("models", "mobilenet_unfrozen_model.h5")

In [66]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint(model_path, monitor='val_loss', 
                     mode='min', verbose=1, save_best_only=True)

In [67]:
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=10,
    callbacks = [es, mc]
)

Epoch 1/30
 39/125 [========>.....................] - ETA: 6:55 - loss: 1.5509 - acc: 0.3355

KeyboardInterrupt: 

Now let's load the best model found:

In [ ]:
# load the saved model
saved_model = load_model(model_path)

## 5.2.4. Validation

Let's plot how the loss and the accuracy from both training and validations sets have evolved during the training process. 

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

In [ ]:
# Plot loss curves
plot_metric_curves(epochs, loss, val_loss, "darkcyan", "turquoise", "Loss")

In [ ]:
# Plot accuracy curves
plot_metric_curves(epochs, acc, val_acc, "darkcyan", "turquoise", "Accuracy")

In [ ]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=input_shape[:2],
        batch_size=40,
        class_mode='categorical',
        shuffle=False
        )

Found 2000 images belonging to 4 classes.


In [ ]:
model.evaluate(test_generator)

In [ ]:
saved_model.evaluate(test_generator)

*Comments about how those metrics have evolved...*